In [ ]:
import pandas as pd
water=pd.read_csv('../input/water-potability/water_potability.csv')
water.head()

In [ ]:
water.info()

In [ ]:
water.describe()

In [ ]:
Potability_0=water[water.Potability==0]
Potability_0.head()

In [ ]:
round(Potability_0.isnull().sum()*100/len(Potability_0),2).sort_values(ascending=False)

In [ ]:
Potability_0.describe()

#### Treating Missing value for Potability_0 records

In [ ]:
Potability_0.fillna(Potability_0.median(), inplace=True)
Potability_0.describe()

In [ ]:
Potability_1=water[water.Potability==1]
Potability_1.head()

In [ ]:
round(Potability_1.isnull().sum()*100/len(Potability_1),2).sort_values(ascending=False)

#### Treating Missing value for Potability_1 records

In [ ]:
Potability_1.fillna(Potability_1.median(), inplace=True)
Potability_1.describe()

In [ ]:
import numpy as np
water=pd.concat([Potability_1, Potability_0], axis=0)
water = water.iloc[np.random.permutation(len(water))]
water=water.reset_index(drop=True)
water.head()

In [ ]:
water.nunique()

In [ ]:
round(water.Potability.value_counts()*100/len(water),2)

In [ ]:
!pip install --upgrade git+https://github.com/tusharsarkar3/XBNet.git

This is based on paper named "XBNet : An Extremely Boosted Neural Network"

Link : https://arxiv.org/abs/2106.05239

In [ ]:
import torch
import numpy as np
from sklearn.model_selection import train_test_split
from XBNet.training_utils import training,predict
from XBNet.models import XBNETClassifier
from XBNet.run import run_XBNET 

In [ ]:
y=water[['Potability']]
y.head()

In [ ]:
x=water.loc[:,:'Turbidity']
x.head()

In [ ]:
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size= 0.20, random_state= True,stratify=y) 
x_train.shape,x_test.shape,y_train.shape,y_test.shape

In [ ]:
from imblearn.over_sampling import SMOTE

sm = SMOTE(random_state=42)
x_train_smote, y_train_smote = sm.fit_resample(x_train, y_train)

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
x_train_smote=scaler.fit_transform(x_train_smote)
x_test=scaler.transform(x_test)

In [ ]:
y_test= y_test.to_numpy()
y_test[:5]

In [ ]:
y_train_smote= y_train_smote.to_numpy()
y_train_smote[:5]

In [ ]:
y_train_smote=y_train_smote.reshape((-1))
y_train_smote.shape

In [ ]:
y_test=y_test.reshape((-1))
y_test.shape

In [ ]:
model = XBNETClassifier(x_train_smote,y_train_smote,num_layers=2)

In [ ]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.04) 

In [ ]:
m,acc, lo, val_ac, val_lo = run_XBNET(x_train_smote,x_test,y_train_smote,y_test,model,criterion,
                                      optimizer,epochs=100,batch_size=256)

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(20,5))
plt.subplot(1,2,1)
plt.plot(acc,label='training accuracy')
plt.plot(val_ac,label = 'validation accuracy')
plt.xlabel('epochs')
plt.ylabel('accuracy')
plt.legend()
plt.grid()
plt.subplot(1,2,2)
plt.plot(lo,label='training loss')
plt.plot(val_lo,label = 'validation loss')
plt.xlabel('epochs')
plt.ylabel('loss')
plt.legend() 
plt.grid()